In [324]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.feature_selection import VarianceThreshold, SelectPercentile, SelectKBest, RFECV, f_regression 
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import  SVR

In [325]:
X_test = pd.read_csv("X_test.csv")
X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")

columns = X_train.columns[1:]

In [326]:
X_train

,id,x0,x1,x2,x3,x4,x5,x6,x7,x8,...,x822,x823,x824,x825,x826,x827,x828,x829,x830,x831
0,0.0,10.891876,832442.812375,20585.544083,1028.369495,1.163780e+06,9.199135,597900.477629,NaN,1.144294e+06,...,1.024198e+06,-855.549602,12176.073427,10.647729,10.916371,1220.065443,8.566724,1.036263e+06,85338.558539,103088.664210
1,1.0,11.512994,832442.898114,NaN,1012.624877,1.028911e+06,10.906408,597900.458612,8127.016078,1.099166e+06,...,1.086806e+06,-787.397942,10493.095660,10.586492,9.463962,917.094909,10.231822,1.007163e+06,95695.020645,105161.109422
2,2.0,11.052185,832442.896307,20585.512844,1003.953827,9.231756e+05,9.212979,597900.426764,10738.092422,1.027863e+06,...,1.018533e+06,-906.997242,10959.516944,10.769287,10.342160,637.027802,10.705461,1.019955e+06,80253.299882,104177.051666
3,3.0,11.642076,NaN,NaN,1004.672084,9.459461e+05,9.553420,597900.450367,13524.096973,1.168144e+06,...,1.047017e+06,-1011.742516,16845.309819,10.483830,10.594941,1114.069590,10.321063,1.085442e+06,NaN,102746.516920
4,4.0,10.407121,832442.831424,20585.557007,NaN,9.957182e+05,8.419164,597900.423639,12894.065081,1.063199e+06,...,1.031009e+06,-1025.223865,18348.460040,NaN,NaN,1230.088215,10.250096,1.024812e+06,101815.745499,105163.749149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207,1207.0,9.895208,832442.883343,20585.510774,1082.657477,1.107019e+06,10.915545,597900.459438,NaN,1.020097e+06,...,1.004860e+06,-999.439794,17689.388714,10.472291,9.792557,1249.098796,10.939983,1.092890e+06,97402.723818,107110.572397
1208,1208.0,9.838322,832442.816133,20585.530631,1020.816077,9.834291e+05,NaN,597900.428561,10842.043309,1.110318e+06,...,1.080810e+06,-1113.072752,15994.070051,10.269221,9.771282,1402.080672,10.958833,1.083262e+06,107100.133295,109984.169649
1209,1209.0,9.267322,832442.815687,20585.525811,1076.663108,7.651645e+05,9.579855,NaN,NaN,1.092330e+06,...,1.047013e+06,-482.079954,11539.602821,10.346105,10.361797,965.008332,10.483549,1.007871e+06,119139.273272,105240.374580
1210,1210.0,NaN,832442.826707,20585.564056,1076.359217,9.332198e+05,8.863739,597900.402740,13121.042555,8.245987e+05,...,1.070566e+06,-1063.653564,15108.683504,10.831122,10.167780,NaN,10.647956,1.047863e+06,104741.008263,100272.537058


In [327]:
# convert to np arrays
X = X_train.iloc[:,1:].to_numpy()
y = y_train.iloc[:,1:].to_numpy().reshape(-1)
X_test = X_test.iloc[:,1:].to_numpy()


In [219]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imputer = IterativeImputer(random_state=0, verbose=1).fit(X)
X = imputer.transform(X)
X_test = imputer.transform(X_test)

[IterativeImputer] Completing matrix with shape (1212, 832)


KeyboardInterrupt: 

In [328]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=15)
X = imputer.fit_transform(X)
X_test = imputer.fit_transform(X_test)

In [329]:
outliers = IsolationForest(n_estimators=1000, random_state=0).fit_predict(X)
X = X[np.where(outliers == 1)[0]].copy()
y = y[np.where(outliers == 1)[0]].copy()

In [330]:
selector = VarianceThreshold()
selector.fit(X)
to_delete = np.where(selector.variances_ == 0)[0]
X = np.delete(X, to_delete, axis=1)
X_test = np.delete(X_test, to_delete, axis=1)

In [331]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)

In [423]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import GradientBoostingRegressor

est = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.1, max_depth=100, random_state=0,loss='squared_error')
r1 = SVR(C=37, gamma=0.0219)
r2 = RandomForestRegressor(n_estimators=1000, random_state=1)
regr = make_pipeline(RobustScaler(), SelectKBest(f_regression, k=94), VotingRegressor([("est", est), ("r1", r1), ("r2", r2)]))
cv_results = cross_validate(regr, X, y, scoring=('r2'))
print(cv_results['test_score'].mean())

0.5629031866702311


In [389]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from keras.layers.advanced_activations import LeakyReLU
from tensorflow import keras
from keras.layers import Dropout
from sklearn.metrics import r2_score
from keras import backend as K

def coeff_determination(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

model = Sequential()
model.add(Dense(256, input_dim=X.shape[1], activation='relu', kernel_initializer='normal'))
model.add(Dense(256, activation='relu', kernel_initializer='normal'))
model.add(Dense(256, activation='relu', kernel_initializer='normal'))
model.add(Dense(256, activation='relu', kernel_initializer='normal'))
model.add(Dense(1, activation='linear'))

# Compile the network :
opt = keras.optimizers.Adam(learning_rate=0.001, clipnorm=1)
model.compile(loss='mean_absolute_error', optimizer=opt, metrics=[coeff_determination])
model.summary()

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_102 (Dense)            (None, 256)               212224    
_________________________________________________________________
dense_103 (Dense)            (None, 256)               65792     
_________________________________________________________________
dense_104 (Dense)            (None, 256)               65792     
_________________________________________________________________
dense_105 (Dense)            (None, 256)               65792     
_________________________________________________________________
dense_106 (Dense)            (None, 1)                 257       
Total params: 409,857
Trainable params: 409,857
Non-trainable params: 0
_________________________________________________________________


In [390]:
scaler = RobustScaler()
scaler.fit(X, y)
xscale = scaler.transform(X)
model.fit(xscale, y, epochs=500, batch_size=32, validation_split = 0.1)


Epoch 1/500
34/34 [==============================] - 1s 7ms/step - loss: 31.9870 - coeff_determination: -19.9648 - val_loss: 11.5529 - val_coeff_determination: -2.7299
Epoch 2/500
34/34 [==============================] - 0s 4ms/step - loss: 10.1055 - coeff_determination: -1.2945 - val_loss: 10.3552 - val_coeff_determination: -0.7919
Epoch 3/500
34/34 [==============================] - 0s 4ms/step - loss: 7.8090 - coeff_determination: -0.1494 - val_loss: 9.6652 - val_coeff_determination: -0.6679
Epoch 4/500
34/34 [==============================] - 0s 4ms/step - loss: 7.2756 - coeff_determination: -0.0183 - val_loss: 9.4280 - val_coeff_determination: -0.6081
Epoch 5/500
34/34 [==============================] - 0s 4ms/step - loss: 6.5417 - coeff_determination: 0.1620 - val_loss: 9.0982 - val_coeff_determination: -0.5687
Epoch 6/500
34/34 [==============================] - 0s 3ms/step - loss: 6.0459 - coeff_determination: 0.2774 - val_loss: 9.7234 - val_coeff_determination: -0.6282
Epoch 7

In [394]:
y_pred = model.predict(scaler.transform(X_test))

In [416]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

scaler = RobustScaler()
scaler.fit(X, y)
xscale = scaler.transform(X)

def baseline_model():
    model = Sequential()
    model.add(Dense(256, input_dim=828, activation='relu'))
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1, activation='linear'))
    opt = keras.optimizers.Adam(clipnorm=1)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=1000, batch_size=100)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, xscale, y, cv=kfold, scoring=('r2'))
print((results.mean(), results.std()))
estimator.fit(xscale, y)

11/11 [==============================] - 1s 12ms/step - loss: 4124.1812
(-16.96624572811162, 10.962645275601497)
13/13 [==============================] - 1s 11ms/step - loss: 2189.3704


In [411]:
y_pred = estimator.predict(scaler.transform(X_test))
y_pred

array([41.3535  , 43.583008, 36.056023, 40.07811 , 44.313328, 51.26306 ,
       39.655518, 42.938553, 53.479298, 65.72723 , 48.047928, 59.113697,
       44.719578, 47.918854, 47.493637, 59.415245, 49.645622, 41.655605,
       39.95689 , 40.01101 , 47.475067, 47.96055 , 38.684395, 39.53775 ,
       35.541313, 46.02296 , 42.637096, 45.01661 , 44.667244, 40.850296,
       40.48659 , 53.936996, 39.22633 , 38.267548, 39.614407, 39.628727,
       45.30624 , 53.304398, 40.02366 , 44.47377 , 39.28413 , 37.31613 ,
       36.285187, 47.67542 , 58.256805, 40.378418, 47.8553  , 40.748547,
       35.402153, 45.345688, 46.174522, 40.028904, 38.739986, 51.29814 ,
       51.47094 , 51.61507 , 43.832115, 41.93601 , 39.82575 , 42.49613 ,
       47.132507, 48.35798 , 38.45488 , 41.357445, 57.43049 , 34.60698 ,
       39.626907, 54.15797 , 31.941652, 46.990814, 48.09926 , 39.837257,
       48.3526  , 42.911854, 49.034214, 40.778435, 67.073494, 35.16932 ,
       48.72271 , 72.332405, 38.562557, 46.02147 , 

In [362]:
regr = make_pipeline(RobustScaler(), SelectKBest(f_regression, k=94), SVR(C=37, gamma=0.0219))
regr = regr.fit(X ,y)
y_pred = regr.predict(X_test)

In [392]:
X_test = pd.read_csv("X_test.csv")
df_ids = pd.DataFrame(X_test['id'])
df_predictions = df_ids.join(pd.DataFrame(y_pred, columns=['y']))

In [393]:
df_predictions.to_csv('predictions.csv', index=False)

In [45]:
from hyperopt import hp, tpe, Trials
from hyperopt.fmin import fmin
import lightgbm as lgb
import hyperopt
import lightgbm
import warnings
warnings.simplefilter("ignore")


random_state=42
n_iter = 500

# trials will contain logging information

best_dict= {}
trials = Trials()
space = {
      'boosting_type': hp.choice('boosting_type',
                                ['gbdt','goss']),
      #dart
      'objective': hp.choice('objective',
                                ['regression','huber']),
      'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
      'colsample_bytree': hp.uniform('colsample_bytree', 0.1,0.9),
      #n_estimators': hp.quniform('n_estimators', 100, 3000, 1),
      'max_depth' : hp.quniform('max_depth', 2, 20, 1),
      'min_child_weight': hp.choice('min_child_weight', np.arange(1, 8, 1, dtype=int)),
      'subsample':        hp.uniform('subsample', 0.5, 1),
      'reg_alpha':        hp.uniform('reg_alpha', 0.001, 1),
      'reg_lambda':        hp.uniform('reg_lambda', 0.001, 1),
      'num_leaves': hp.quniform('num_leaves', 10, 120, 1),
      'n_rows': hp.quniform('n_rows', 1000, len(X_train), 1),
  }

def gb_mse_cv(params, random_state=random_state):

    print(params)
    params['task'] = 'train'
    params['metric'] = 'l2'
    params['verbose'] = -1
    params['verbose_eval'] = -1

    n_rows = int(params.pop('n_rows'))
    #params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])
    params['num_leaves'] = int(params['num_leaves'])

    train_data = lgb.Dataset(X_train[-n_rows:], label=y_train[-n_rows:],free_raw_data=False)
    valid_data = lgb.Dataset(X_test, label=y_test,free_raw_data=False)
    gbm = lgb.train(params,train_data,valid_sets=(valid_data,train_data),early_stopping_rounds=100,verbose_eval=False)
    score = gbm.best_score['valid_0']['l2']
    print(score)
    return score


best=fmin(fn=gb_mse_cv, # function to optimize
          space=space, 
          algo=tpe.suggest, # optimization algorithm, hyperotp will select its parameters automatically
          max_evals=n_iter, # maximum number of iterations
          trials=trials, # logging
          rstate=np.random.RandomState(random_state) # fixing random state for the reproducibility
        )
best['boosting_type'] = ['gbdt','goss'][best['boosting_type']]
best['objective'] = ['regression','huber'][best['objective']]
best['max_depth'] = int(best['max_depth'])
best['n_estimators'] = int(best['n_estimators'])
best['num_leaves'] = int(best['num_leaves'])
best 

{'boosting_type': 'gbdt', 'colsample_bytree': 0.2409770452841129, 'learning_rate': 0.016479447147000035, 'max_depth': 6.0, 'min_child_weight': 4, 'n_rows': 1061.0, 'num_leaves': 38.0, 'objective': 'huber', 'reg_alpha': 0.0699822895327306, 'reg_lambda': 0.8327994002826448, 'subsample': 0.7273505261106861}
[LightGBM] [Warning] Unknown parameter: verbose_eval
70.67710618807371
{'boosting_type': 'goss', 'colsample_bytree': 0.7933417537676787, 'learning_rate': 0.016121587285596147, 'max_depth': 10.0, 'min_child_weight': 3, 'n_rows': 1004.0, 'num_leaves': 114.0, 'objective': 'huber', 'reg_alpha': 0.9329351308489858, 'reg_lambda': 0.7279980174341476, 'subsample': 0.7050519794266569}
70.12678890637851
{'boosting_type': 'gbdt', 'colsample_bytree': 0.6085832815033362, 'learning_rate': 0.028842161903625093, 'max_depth': 4.0, 'min_child_weight': 3, 'n_rows': 1045.0, 'num_leaves': 72.0, 'objective': 'regression', 'reg_alpha': 0.3041066997742908, 'reg_lambda': 0.8834319621034812, 'subsample': 0.9186

KeyError: 'n_estimators'

In [49]:
best

{'boosting_type': 'goss',
 'colsample_bytree': 0.36057252477815943,
 'learning_rate': 0.0527194033183651,
 'max_depth': 19,
 'min_child_weight': 3,
 'n_rows': 1082.0,
 'num_leaves': 15,
 'objective': 'regression',
 'reg_alpha': 0.43260069114870237,
 'reg_lambda': 0.5776124363315015,
 'subsample': 0.8990251985855349}